Using Keras, SELU multilayer perceptrons and Hyperopt on Tox21
====


In [ ]:
import numpy as np
import pandas as pd
from scipy import io
    
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers.noise import AlphaDropout
from keras import optimizers
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

from hyperopt import fmin, tpe, hp, STATUS_OK, STATUS_FAIL, Trials

from sklearn.metrics import roc_auc_score
import sklearn.utils 

import tempfile

import tensorflow as tf

In [ ]:
# All target assays: ['NR.AhR', 'NR.AR', 'NR.AR.LBD', 'NR.Aromatase', 'NR.ER', 'NR.ER.LBD',
#       'NR.PPAR.gamma', 'SR.ARE', 'SR.ATAD5', 'SR.HSE', 'SR.MMP', 'SR.p53']

def data(target):
    """
    Load training and test data for a given Tox21 target assay
    """
    source_directory = "/media/matthias/Big-disk/tox21-hochreiter/"

    # load dense features and labels
    y_tr = pd.read_csv(source_directory + 'tox21_labels_train.csv.gz', index_col=0, compression="gzip")
    y_te = pd.read_csv(source_directory + 'tox21_labels_test.csv.gz', index_col=0, compression="gzip")
    x_tr_dense = pd.read_csv(source_directory + 'tox21_dense_train.csv.gz', index_col=0, compression="gzip").values
    x_te_dense = pd.read_csv(source_directory + 'tox21_dense_test.csv.gz', index_col=0, compression="gzip").values
    
    # load sparse features
    x_tr_sparse = io.mmread(source_directory + 'tox21_sparse_train.mtx.gz').tocsc()
    x_te_sparse = io.mmread(source_directory + 'tox21_sparse_test.mtx.gz').tocsc()
    
    # filter out very sparse features. combine filtered sparse features with dense features
    sparse_col_idx = ((x_tr_sparse > 0).mean(0) > 0.05).A.ravel()
    x_tr = np.hstack([x_tr_dense, x_tr_sparse[:, sparse_col_idx].A])
    x_te = np.hstack([x_te_dense, x_te_sparse[:, sparse_col_idx].A])
    
    # filter out data where no results for target assay are available    
    rows_tr = np.isfinite(y_tr[target]).values
    rows_te = np.isfinite(y_te[target]).values
    x_train = x_tr[rows_tr] 
    y_train = y_tr[target][rows_tr]
    x_test = x_te[rows_te]
    y_test = y_te[target][rows_te]   
    
    return x_train, y_train, x_test, y_test


# We focus on target assay 'NR.AhR' for a start
x_train, y_train, x_test, y_test = data('NR.AhR')

In [3]:
space = {
    'shape': hp.choice('shape', ['square', 'pyramid']),
    'hidden layers': hp.quniform('hidden layers', 0, 10, 1),
    'hidden units': hp.quniform('hidden units', 200, 2048, 1),
    'dropout rate': hp.uniform('dropout rate', .001, .5),
    'batch size' : hp.quniform('batch size', 10, 100, 1),
    'learning rate': hp.uniform('learning rate', 1e-06, 1e-04 ),
    'optimizer': hp.choice('optimizer',['sgd','adam']),
    'kernel initializer': hp.choice('kernel initializer',['lecun_normal','he_normal', 'glorot_uniform']),
    'l2 kernel regularization': hp.uniform('l2 kernel regularization', 1e-06, 1e-05)}

def f_nn(p):
    print("\n")
    print(p)
    
    try:
        max_epochs = 100

        # some parameters need to be cast to integer
        p['hidden layers'] = int(p['hidden layers'])
        p['hidden units'] = int(p['hidden units'])
        p['batch size'] = int(p['batch size'])

        number_of_features = x_train.shape[1]
        
        # clear session (to avoid clutter from old models)
        K.clear_session()
        
        # the model is a classical multilayer perceptron with SELU activations and Alpha dropout
        model = Sequential()
        model.add(Dense(p['hidden units'], input_dim=number_of_features, 
                        activation='selu', kernel_initializer=p['kernel initializer'], 
                        kernel_regularizer=regularizers.l2(p['l2 kernel regularization'])))
        for i in range(p['hidden layers']):
            if p['shape'] == 'pyramid':
                # pyramid shape: each layer has half the units of the previous layer (but at least 2 units)
                number_of_units_in_this_layer = max(int(p['hidden units'] / pow(2, i+1)), 2) 
            else:
                number_of_units_in_this_layer = p['hidden units']
            model.add(AlphaDropout(p['dropout rate']))
            model.add(Dense(p['hidden units'], activation='selu', kernel_initializer=p['kernel initializer'],
                           kernel_regularizer=regularizers.l2(p['l2 kernel regularization'])))
        model.add(Dense(1, activation='sigmoid'))

        if p['optimizer'] == 'sgd':
            optimizer=optimizers.sgd(lr=p['learning rate'])
        if p['optimizer'] == 'adam':
            optimizer=optimizers.adam(lr=p['learning rate'])


        model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])

        # weigh classes (in case labels are unbalanced)
        class_weight = sklearn.utils.class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

        _, tmpfn = tempfile.mkstemp()

        callbacks = [EarlyStopping(patience=15), ModelCheckpoint(tmpfn, save_best_only=True, save_weights_only=True)]
        model.fit(x_train, y_train, epochs=max_epochs, batch_size=p['batch size'], validation_data=(x_test, y_test), 
              callbacks=callbacks, class_weight=class_weight, verbose=0)

        # load model weights of epoch that had best results
        model.load_weights(tmpfn)

        p_te = model.predict_proba(x_test)

        # add column for other class, adding up to probability of 1
        new_col = np.subtract(1,p_te).reshape((p_te.shape[0],1))
        p_te_both_classes = np.append(new_col,p_te, 1)

        auc_te = roc_auc_score(y_test, p_te_both_classes[:, 1])
        print("%15s score on test: %3.5f" % (target, auc_te))

        return {'loss': -auc_te, 'status': STATUS_OK, 'model': model}
    
    except tf.errors.ResourceExhaustedError: 
        # fail gracefully if model is too large to fit into memory (so hyperparameter search can continue)
        print("ResourceExhaustedError occurred, skipping...")
        return {'status': STATUS_FAIL}

trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=50, trials=trials)

print('best: ')
print(best)



{'batch size': 34.0, 'dropout rate': 0.07970398031653203, 'hidden layers': 6.0, 'hidden units': 1439.0, 'kernel initializer': 'he_normal', 'l2 kernel regularization': 4.570848329984473e-06, 'learning rate': 1.7760650984633667e-05, 'optimizer': 'adam', 'shape': 'pyramid'}
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.82197


{'batch size': 20.0, 'dropout rate': 0.07149632323164115, 'hidden layers': 9.0, 'hidden units': 1999.0, 'kernel initializer': 'glorot_uniform', 'l2 kernel regularization': 5.341642875118799e-06, 'learning rate': 7.128224107068452e-05, 'optimizer': 'sgd', 'shape': 'square'}
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.78562


{'batch size': 65.0, 'dropout rate': 0.4171895094613189, 'hidden layers': 8.0, 'hidden units': 1057.0, 'kernel initializer': 'lecun_normal', 'l2 kernel regularization': 1.7659788690358359e-06, 'learning rate': 5.815697560092928e-05, 'optimizer': 'sgd', 'shape': 'pyramid

 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.81008


{'batch size': 43.0, 'dropout rate': 0.24371606269248938, 'hidden layers': 3.0, 'hidden units': 783.0, 'kernel initializer': 'he_normal', 'l2 kernel regularization': 3.561382036885765e-06, 'learning rate': 2.0922619527506757e-05, 'optimizer': 'adam', 'shape': 'pyramid'}
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.79720


{'batch size': 52.0, 'dropout rate': 0.02455922161449304, 'hidden layers': 5.0, 'hidden units': 1325.0, 'kernel initializer': 'glorot_uniform', 'l2 kernel regularization': 6.076351534048634e-06, 'learning rate': 3.3000701278864124e-05, 'optimizer': 'adam', 'shape': 'pyramid'}
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.46741


{'batch size': 53.0, 'dropout rate': 0.2806510239225377, 'hidden layers': 3.0, 'hidden units': 881.0, 'kernel initializer': 'he_normal', 'l2 kernel regularization': 4.234249370626

 32/610 [>.............................] - ETA: 1s         NR.AhR score on test: 0.56843


{'batch size': 48.0, 'dropout rate': 0.4519645165295243, 'hidden layers': 7.0, 'hidden units': 252.0, 'kernel initializer': 'glorot_uniform', 'l2 kernel regularization': 8.335776641085155e-06, 'learning rate': 6.862756000386326e-05, 'optimizer': 'adam', 'shape': 'square'}
 32/610 [>.............................] - ETA: 1s         NR.AhR score on test: 0.80679


{'batch size': 62.0, 'dropout rate': 0.07647742589205056, 'hidden layers': 9.0, 'hidden units': 988.0, 'kernel initializer': 'lecun_normal', 'l2 kernel regularization': 5.794125792648407e-06, 'learning rate': 4.597745832972281e-05, 'optimizer': 'sgd', 'shape': 'square'}
 32/610 [>.............................] - ETA: 0s         NR.AhR score on test: 0.77567


{'batch size': 29.0, 'dropout rate': 0.3424094236187401, 'hidden layers': 6.0, 'hidden units': 574.0, 'kernel initializer': 'glorot_uniform', 'l2 kernel regularization': 8.01652509194

Train and evaluate model with best hyperparameters
----------

In [10]:
from hyperopt import space_eval 
training_result = f_nn(space_eval(space, best))



{'batch size': 35.0, 'dropout rate': 0.029867293022818375, 'hidden layers': 10.0, 'hidden units': 419.0, 'kernel initializer': 'glorot_uniform', 'l2 kernel regularization': 7.0412947425240335e-06, 'learning rate': 5.5633772459955735e-05, 'optimizer': 'adam', 'shape': 'square'}
 32/610 [>.............................] - ETA: 1s         NR.AhR score on test: 0.83457


Calculate ROC AUC on *training* data to see if we are over- or under-fitting
-----

In [15]:
model = training_result['model']
for target in ['NR.AhR']:    
    p_tr = model.predict_proba(x_train)
    # add column for other class, adding up to probability of 1
    new_col = np.subtract(1,p_tr).reshape((p_tr.shape[0],1))
    p_tr_both_classes = np.append(new_col,p_tr, 1)
    
    auc_tr = roc_auc_score(y_train, p_tr_both_classes[:, 1])
    print("%15s score on training: %3.5f" % (target, auc_tr))

7680/8441 [==========================>...] - ETA: 0s         NR.AhR score on training: 0.81494


Seems like we are not overfitting